# Preliminary EDA: West Coast Datathon 2020

The goal of this notebook is to search for answerable questions in available data sets as well as to identify useful supplemental data sets which are available online.

Author: Marlin Figgins

Date: Sept. 28, 2018.